In [1]:
import numpy as np
from pandas import read_csv
from pandas import to_datetime
from sklearn.preprocessing import MinMaxScaler
import math

# load the train data
dataframe = read_csv('dataset/train.csv',usecols=[i for i in range(0,211)], engine='python', header=0)
trainData = dataframe.values
trainDataOriginal = np.nan_to_num(trainData)
trainData = trainDataOriginal
# load the test data
dataframe = read_csv('dataset/test_2.csv',usecols=[i for i in range(0,146)], engine='python', header=0)
testData = dataframe.values
testData = np.nan_to_num(testData)
testX = testData

In [2]:
#parameters
batch_size = 64
input_dim  = 146
output_dim = 64

In [3]:
#Separate the train data to input and output
trainX = trainData[0:len(trainData),0:146]
trainY = trainData[0:len(trainData),147:211]
# normalize the dataset
scalerInput = MinMaxScaler(feature_range=(0, 1))
scalerOutput = MinMaxScaler(feature_range=(0, 1))

trainX = scalerInput.fit_transform(trainX)
trainY = scalerOutput.fit_transform(trainY)
testX = scalerInput.fit_transform(testX)

#Put it into batches
trainX = np.reshape(trainX,(len(trainX)/batch_size,batch_size,trainX.shape[1]))
trainY = np.reshape(trainY,(len(trainY)/batch_size,batch_size,trainY.shape[1]))
testX  = np.reshape(testX,(len(testX)/batch_size,batch_size,testX.shape[1]))

print trainX.shape
print trainY.shape
print testX.shape

(625, 64, 146)
(625, 64, 64)
(1875, 64, 146)


In [4]:
from keras.activations import linear
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import adam
from sklearn.metrics import mean_squared_error
from keras.models import Model

class Model:
    
    def __init__(self, trainX, trainY):
        self.optimizer = adam(0.01, 0.01, 0.001, 1e-08, 0.0)
        self.model = Sequential()
        self.model.add(LSTM(output_dim, input_shape=(batch_size, input_dim), return_sequences=True))
        self.model.add(Dense(output_dim, activation='linear'))
        #Configures the model for training.
        self.model.compile(loss="mean_squared_error",optimizer=self.optimizer)
        #Trains the model for a fixed number of epochs (iterations on a dataset).
        self.model.fit(trainX,trainY,batch_size=batch_size,epochs=100)
        #Make predictions
    
    def predict(self,testX):
        return self.model.predict(testX)
    

Using TensorFlow backend.


In [5]:
#train the model
nn = Model(trainX, trainY)
testY = nn.predict(testX)

Epoch 1/100
625/625 [==============================] - 2s - loss: 0.0938     
Epoch 2/100
625/625 [==============================] - 1s - loss: 0.0271     
Epoch 3/100
625/625 [==============================] - 1s - loss: 0.0234     
Epoch 4/100
625/625 [==============================] - 1s - loss: 0.0218     
Epoch 5/100
625/625 [==============================] - 1s - loss: 0.0214     
Epoch 6/100
625/625 [==============================] - 1s - loss: 0.0200     
Epoch 7/100
625/625 [==============================] - 2s - loss: 0.0191     
Epoch 8/100
625/625 [==============================] - 1s - loss: 0.0181     
Epoch 9/100
625/625 [==============================] - 1s - loss: 0.0184     
Epoch 10/100
625/625 [==============================] - 1s - loss: 0.0217     
Epoch 11/100
625/625 [==============================] - 2s - loss: 0.0203     
Epoch 12/100
625/625 [==============================] - 2s - loss: 0.0196     
Epoch 13/100
625/625 [==============================] - 1s - 

In [6]:
testY = testY.reshape(len(testData),output_dim)
testY = scalerOutput.inverse_transform(testY)

[[[ 0.27722538  0.41191441  0.25778103 ...,  0.39068079  0.22185877
    0.18526031]
  [ 0.30170268  0.4142175   0.21979235 ...,  0.3276042   0.16439402
    0.1669545 ]
  [ 0.28268045  0.42811126  0.23159155 ...,  0.35434252  0.1559689
    0.17481042]
  ..., 
  [ 0.28803959  0.42057824  0.21537572 ...,  0.33112931  0.17314607
    0.17330882]
  [ 0.28497991  0.41929254  0.2179479  ...,  0.32386723  0.15691514
    0.17446931]
  [ 0.28286085  0.42736083  0.21171799 ...,  0.3156395   0.14905316
    0.17400275]]

 [[ 0.26867196  0.41901064  0.25016084 ...,  0.3912757   0.2162506
    0.17333689]
  [ 0.29639587  0.41994682  0.22336958 ...,  0.35169125  0.17085397
    0.17246018]
  [ 0.28630573  0.43777227  0.22766322 ...,  0.34057897  0.16546756
    0.17940933]
  ..., 
  [ 0.28171715  0.41220164  0.22205271 ...,  0.34943539  0.17107639
    0.17578305]
  [ 0.29075569  0.41770026  0.22600949 ...,  0.34467626  0.16993889
    0.18187602]
  [ 0.2763671   0.43220574  0.22291861 ...,  0.34962893  0.1

In [8]:
import gzip
#create the result.cs.gzip
content = "Id,Predicted\n"
for y in range(testY.shape[0]):
    for x in range(testY.shape[1]-2):
       content +=str(y+1)+"_"+str(x+1)+","+str(testY[y][x])+"\n"

with gzip.open('./result.csv.gz', 'wb') as f:
    f.write(content)